In [25]:
import pandas as pd
df = pd.read_csv('2016_US_County_Level_Presidential_Results.csv')
df.head()

,Unnamed: 0,votes_dem,votes_gop,total_votes,per_dem,per_gop,diff,per_point_diff,state_abbr,county_name,combined_fips
0,0,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2013
1,1,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2016
2,2,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2020
3,3,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2050
4,4,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2060


In [7]:
df = df[['votes_dem','votes_gop','total_votes','per_dem','per_gop','state_abbr','county_name','combined_fips']]

from sqlalchemy import create_engine
engine = create_engine('postgresql://pascalrettig:@localhost:5432/politics')
df.to_sql("results", engine,if_exists="replace")

In [ ]:
engine.execute("""ALTER TABLE counties ADD votes_dem int, ADD votes_gop int, 
                                       ADD total_votes int, ADD differential double precision""")

In [24]:
engine.execute("""UPDATE counties SET votes_dem=results.votes_dem, 
                                      votes_gop=results.votes_gop,
                                      total_votes=results.total_votes 
                                  FROM results 
                                  WHERE combined_fips = to_number(CONCAT(statefp,countyfp),'99999')""")

engine.execute("""UPDATE counties SET differential = 
                  cast((votes_dem - votes_gop) as float) / total_votes""")